# Global-scale atmospheric energy budgets on ERA5 pressure level data

In [1]:
import numpy as np
import xarray as xr

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

ERA5 pressure level data:

* `/glade/derecho/scratch/ksha/CREDIT_data/ERA5_plevel_base/upper_air/*.zarr`
* `/glade/derecho/scratch/ksha/CREDIT_data/ERA5_plevel_base/surf/*.zarr`
* `/glade/derecho/scratch/ksha/CREDIT_data/ERA5_plevel_base/accum/*.zarr`

In [3]:
base_dir = '/glade/derecho/scratch/ksha/CREDIT_data/ERA5_plevel_base/'
ds_surf = xr.open_zarr(base_dir + 'surf/ERA5_plevel_6h_surf_1979.zarr')
ds_accum = xr.open_zarr(base_dir + 'accum/ERA5_plevel_6h_accum_1979.zarr')
ds_upper = xr.open_zarr(base_dir + 'upper_air/ERA5_plevel_6h_upper_air_1979.zarr')
ds_static = xr.open_zarr(base_dir + 'static/ERA5_plevel_6h_static.zarr')

In [16]:
t0 = 200
t1 = 201

In [17]:
GRAVITY = 9.80665
LH_WATER = 2.26e6  # J/kg
CP_DRY = 1005 # J/kg K
CP_VAPOR = 1846 # J/kg K
R = 6371000  # m

In [18]:
x = ds_surf['longitude']
y = ds_surf['latitude']
lon, lat = np.meshgrid(x, y)
level_p = 100*np.array(ds_upper['level'])

In [19]:
level_p # Pa or kg/m/s2

array([   100.,    200.,    300.,    500.,    700.,   1000.,   2000.,
         3000.,   5000.,   7000.,  10000.,  12500.,  15000.,  17500.,
        20000.,  22500.,  25000.,  30000.,  35000.,  40000.,  45000.,
        50000.,  55000.,  60000.,  65000.,  70000.,  75000.,  77500.,
        80000.,  82500.,  85000.,  87500.,  90000.,  92500.,  95000.,
        97500., 100000.])

In [20]:
q = np.array(ds_upper['Q'].isel(time=slice(t0, t1+1))) # kg/kg
T = np.array(ds_upper['T'].isel(time=slice(t0, t1+1))) # K
u = np.array(ds_upper['U'].isel(time=slice(t0, t1+1))) # m/s
v = np.array(ds_upper['V'].isel(time=slice(t0, t1+1)))
Z = np.array(ds_upper['Z'].isel(time=slice(t0, t1+1)))
GPH_surf = np.array(ds_static['geopotential_at_surface'])[None, None, ...] # J/m2
TOA_net = np.array(ds_accum['top_net_solar_radiation'].isel(time=slice(t0, t1+1))) # J/m2
OLR = np.array(ds_accum['top_net_thermal_radiation'].isel(time=slice(t0, t1+1))) # J/m2
R_short = np.array(ds_accum['surface_net_solar_radiation'].isel(time=slice(t0, t1+1))) # J/m2
R_long = np.array(ds_accum['surface_net_thermal_radiation'].isel(time=slice(t0, t1+1))) # J/m2
LH = np.array(ds_accum['surface_latent_heat_flux'].isel(time=slice(t0, t1+1))) # J/m2
SH = np.array(ds_accum['surface_sensible_heat_flux'].isel(time=slice(t0, t1+1))) # J/m2

In [21]:
def weighted_sum(data, weights, axis, keepdims=False):
    '''
    Compute the weighted sum of a given quantity

    Args:
        data: the quantity to be sum-ed
        weights: weights that can be broadcasted to the shape of data
        axis: dims to compute the sum
        keepdims: keepdims

    Returns:
        weighted sum
    '''
    expanded_weights = np.broadcast_to(weights, data.shape)
    return np.sum(data * expanded_weights, axis=axis, keepdims=keepdims)

def pressure_integral(q, level_p, output_shape):
    '''
    Compute the pressure level integral of a given quantity using np.trapz

    Args:
        q: the quantity with dims of (level, lat, lon) or (time, level, lat, lon)
        level_p: the pressure level of q as [Pa] and with dims of (level,)
        output_shape: either (lat, lon) or (time, lat, lon)

    Returns:
        Pressure level integrals of q
    '''
    # (level, lat, lon) --> (lat, lon)
    if len(output_shape) == 2:
        Q = np.empty(output_shape)
        for ix in range(output_shape[0]):
            for iy in range(output_shape[1]):
                Q[ix, iy] = np.trapz(q[:, ix, iy], level_p)
                
    # (time, level, lat, lon) --> (time, lat, lon)
    elif len(output_shape) == 3:
        Q = np.empty(output_shape)
        for i_time in range(output_shape[0]):
            for ix in range(output_shape[1]):
                for iy in range(output_shape[2]):
                    Q[i_time, ix, iy] = np.trapz(q[i_time, :, ix, iy], level_p)
                    
    else:
        print('wrong output_shape')
        raise
        
    return Q
    
def dx_dy(lat, lon):
    '''
    Compute the grid spacing from 2D lat/lon grids using central difference
    for center grids and forward/backward difference for edge grids

    Args:
        lat, lon: 2D arrays of latitude and longitude.

    Return:
        dy, dx: 2D arrays of grid spacings
    '''
    
    # Convert latitude and longitude from degrees to radians
    lat_rad = np.radians(lat)
    lon_rad = np.radians(lon)
    
    # Compute the grid spacing in the latitude direction (dy)
    dy = np.zeros_like(lat)
    dy[1:-1, :] = R * (lat_rad[2:, :] - lat_rad[:-2, :]) / 2.0
    dy[0, :] = R * (lat_rad[1, :] - lat_rad[0, :])
    dy[-1, :] = R * (lat_rad[-1, :] - lat_rad[-2, :])
    
    # Compute the grid spacing in the longitude direction (dx)
    dx = np.zeros_like(lon)
    dx[:, 1:-1] = R * np.cos(lat_rad[:, 1:-1]) * (lon_rad[:, 2:] - lon_rad[:, :-2]) / 2.0
    dx[:, 0] = R * np.cos(lat_rad[:, 0]) * (lon_rad[:, 1] - lon_rad[:, 0])
    dx[:, -1] = R * np.cos(lat_rad[:, -1]) * (lon_rad[:, -1] - lon_rad[:, -2])

    return dy, dx

def compute_grid_area(lat, lon):
    '''
    Compute grid cell areas from 2D lat/lon grids

    Args:
        lat, lon: latitude and longitude with dims of (lat, lon)

    Return:
        grid cell area, dims are (lat, lon)
    '''
    dy, dx = dx_dy(lat, lon)
    area = dy*dx

    return area

area = np.abs(compute_grid_area(lat, lon))

# w_lat = np.cos(np.deg2rad(lat))
w_lat = area #/ np.sum(area)

## Conservation of energy

Note: (1) clould processes and (2) frictional dissipation are not considered

Reference:
* [Trenberth and Solomon (1994)](https://link.springer.com/content/pdf/10.1007/BF00210625.pdf)
* [Trenberth and Fasullo (2018)](https://doi.org/10.1175/JCLI-D-17-0838.1)
* [Atmospheric conservation properties
in ERA-Interim](https://www.ecmwf.int/sites/default/files/elibrary/2011/8175-atmospheric-conservation-properties-era-interim.pdf)
* [ERA5 variable table](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview)
* [CAM6 Global Energy Fixer](https://ncar.github.io/CAM/doc/build/html/cam6_scientific_guide/dynamics.html#fvmap)

Can you summarize the key points of correction plan 1 using the following equation notations:

Equation on a single air-column:

\begin{equation}
\frac{\partial \mathrm{E}}{\partial t} + \mathbf{\nabla}\cdot\frac{1}{g}\int_{p_0}^{p_1}{\mathbf{v}\left(h+k\right)}dp = R_T - F_S
\end{equation}

$E$ is the total energy of an air column

\begin{equation}
E = \frac{1}{g}\int_{p_0}^{p_1}{\left(C_pT+Lq+\Phi_s+k\right)}dp
\end{equation}

These terms are (1) thermal energy, (2) latent heat energy, (3) gravitational potential energy, and (4) kinetic energy. $L$ is latent heat of condensation of water, $C_p$ is  the specific heat capacity of air at constant pressure, $\Phi_s$ is geopotential at surface, kinetic energy is defined as $k=0.5*\left(\mathbf{v} \cdot \mathbf{v}\right)$.

$C_p$ is a function of humidity:

\begin{equation}
C_p = (1-q) C_{pd} + q C_{pv}
\end{equation}

$R_T$ and $F_S$ are energy fluxes on the top of the atmosphere and the surface of earth:

\begin{equation}
R_T = TOA_{\mathrm{net}} + OLR
\end{equation}

\begin{equation}
F_s = R_{\mathrm{short}} + R_{\mathrm{long}} + H_{\mathrm{sensible}} + H_{\mathrm{latent}}
\end{equation}

For global sum, the divergence term is 0, so the time tendency of total energy is balanced by energy fluxes from the top of the atmosphere and the surface of earth:

\begin{equation}
\overline{\left(\frac{\partial E}{\partial t}\right)} =  \overline{R_T} - \overline{F_s}
\end{equation}

When the balance above is not conserved, $T$ can be adjusted to close the budget.

Given time $t_0$ and $t_1$, the time tendency of total energy per unit area is:

\begin{equation}
   \frac{\partial \mathrm{E}}{\partial t} \sim \frac{\Delta E}{\Delta t} = \frac{E_{t_1} - E_{t_0}}{\Delta t}
\end{equation}

Where $\Delta t$ is the time interval between $t_0$ and $t_1$ as seconds.

Given the net energy flux into the air column from top of atmosphere and surface, the global sum of energy-conserved total energy $E^*$ at time $t_1$ is:

\begin{equation}
   \overline{E_{t_1}^*} = \overline{E_{t_0}} + {\Delta t}\left(\overline{R_T} - \overline{F_s}\right)
\end{equation}

Correction ratio $\gamma$ can be computed between the energy-conserved $\overline{E_{t_1}^*}$ and the original $\overline{E_{t_1}}$

\begin{equation}
   \gamma = \frac{\overline{E_{t_1}^*}}{\overline{E_{t_1}}}
\end{equation}

$\gamma$ is a constant value that represents the averaged correction of all atmospheric columns. In this study, $\gamma$ is used for unified sptial correction of all grid cells and barotropic correction of all pressure levels, which leads to the correction air temperature that can close the energy conservation budget:

\begin{equation}
   T_{t_1}^* = \gamma\ T_{t_1} + \frac{\gamma-1}{C_p}\left(Lq_{t_1}+\Phi_s+k_{t_1}\right)
\end{equation}

In [22]:
# A toy correction model to clearify the logic
# # Original data
# a_t0 = np.array([[10, 20, 30, 10, 5], [15, 25, 35, 15, 10]])
# f_a_t0 = np.array([0.8, 0.8, 0.9, 0.7, 0.6])

# a_t1 = np.array([[11, 21, 31, 12, 4], [16, 24, 38, 19, 9]])
# f_a_t1 = np.array([0.81, 0.81, 0.91, 0.72, 0.61])

# b = np.array([[5, 10, 15, 20, 25], [6, 11, 16, 19, 24]])

# w = np.array([150, 100])
# y_level = np.array([100, 200, 300, 700, 1000])

# # Calculate initial energies
# E_t0 = f_a_t0 * a_t0 + b
# E_t1 = f_a_t1 * a_t1 + b

# # Calculate total energies by integrating over y_level
# TE_t0 = np.trapz(E_t0, y_level)  # Shape: (2,)
# TE_t1 = np.trapz(E_t1, y_level)  # Shape: (2,)

# # Calculate the rate of change of total energy
# dTE_dt = (TE_t1 - TE_t0) / 60

# # Calculate weighted sum of energy change
# dTE_sum = np.sum(dTE_dt * w)

# # Desired corrected total energy change
# dTE_sum_correct = 6000

# # Calculate total weighted energies
# total_weighted_TE_t0 = np.sum(TE_t0 * w)
# total_weighted_TE_t1 = np.sum(TE_t1 * w)

# # Calculate the correction ratio for TE_t1
# E_correct_ratio = (60 * dTE_sum_correct + total_weighted_TE_t0) / total_weighted_TE_t1

# # Apply the correction factor only to E_t1
# E_t1_correct = E_t1 * E_correct_ratio

# # Recalculate a_t1_correct from corrected E_t1
# a_t1_correct = (E_t1_correct - b) / f_a_t1

# # Recalculate TE_t1 with corrected E_t1
# TE_t1_correct = np.trapz(E_t1_correct, y_level)

# # Recalculate the rate of change of total energy
# dTE_dt = (TE_t1_correct - TE_t0) / 60

# # Recalculate the corrected dTE_sum
# dTE_sum = np.sum(dTE_dt * w)

# # Compute the error
# error = dTE_sum_correct - dTE_sum

# print("Corrected dTE_sum:", dTE_sum)
# print("Error after correction:", error)

In [23]:
N_seconds = 3600 * 6
# min_ratio = 0.9     # Minimum allowable correction ratio
# max_ratio = 1.1     # Maximum allowable correction ratio

In [27]:
C_p = (1-q)*CP_DRY + q*CP_VAPOR
ken = 0.5*(u**2 + v**2)
T_correct = np.copy(T)

# TOA energy flux
R_T = (TOA_net + OLR) / N_seconds
R_T = R_T[1, ...]
R_T_sum = weighted_sum(R_T, w_lat, axis=(0, 1), keepdims=False)

# surface energy flux
F_S = (R_short + R_long + LH + SH) / N_seconds
F_S = F_S[1, ...]
F_S_sum = weighted_sum(F_S, w_lat, axis=(0, 1), keepdims=False)

# latent heat energy, static energy, kinetic energy fluxes
E_qgk = LH_WATER*q + GPH_surf + ken
# E_qgk = LH_WATER*q + Z + ken

correction_cycle_num = 1

for i in range(correction_cycle_num):
    
    E_level = C_p * T_correct + E_qgk
    TE = pressure_integral(E_level, level_p, (2,)+lon.shape) / GRAVITY
    
    dTE_dt = (TE[1, ...] - TE[0, ...]) / N_seconds
    dTE_sum = weighted_sum(dTE_dt, w_lat, axis=(0, 1), keepdims=False)
    
    delta_dTE_sum = (R_T_sum - F_S_sum) - dTE_sum 
    
    print('Residual to conserve energy budge [Watts]: {}'.format(delta_dTE_sum))

    # ====== #
    # Plan 1 # <--------- correction plan 1
    # TE_t0 = TE[0, ...]
    # TE_t1 = TE[1, ...]
    # adjusted_TE_t1 = N_seconds * (R_T - F_S) + TE_t0
    # E_correct_ratio = adjusted_TE_t1 / TE_t1
    # E_correct_ratio = np.clip(E_correct_ratio, min_ratio, max_ratio)
    # E_level[1, ...] = E_correct_ratio * E_level[1, ...]
    # T_correct[1, ...] = (E_level[1, ...] - E_qgk[1, ...]) / C_p[1, ...]

    # ====== #
    # Plan 2 # <--------- correction plan 2
    total_weighted_TE_t0 = weighted_sum(TE[0, ...], w_lat, axis=(0, 1), keepdims=False)
    total_weighted_TE_t1 = weighted_sum(TE[1, ...], w_lat, axis=(0, 1), keepdims=False)
    
    # Calculate the correction ratio for E_t1
    E_correct_ratio = (N_seconds * (R_T_sum - F_S_sum) + total_weighted_TE_t0) / total_weighted_TE_t1
    
    E_t1_correct = E_level[1, ...] * E_correct_ratio
    T_correct[1, ...] = (E_t1_correct - E_qgk[1, ...]) / C_p[1, ...]

    # ====== #
    # Plan 3 # <--------- correction plan 3
    # E_correct_ratio = (R_T_sum - F_S_sum) / dTE_sum
    # E_level_correct = E_level * E_correct_ratio
    # T_correct_approx = ((E_level_correct - E_qgk) / C_p)
    # T_correct[1, ...] = T_correct_approx[1, ...] - T_correct_approx[0, ...] + T_correct[0, ...]
    
E_level = C_p * T_correct + E_qgk
TE = pressure_integral(E_level, level_p, (2,)+lon.shape) / GRAVITY

dTE_dt = (TE[1, ...] - TE[0, ...]) / N_seconds
dTE_sum = weighted_sum(dTE_dt, w_lat, axis=(0, 1), keepdims=False)

energy_residual = dTE_sum - (R_T_sum - F_S_sum)
print('Residual to conserve energy budge [Watts]: {}'.format(energy_residual))

Residual to conserve energy budge [Watts]: 1026453655660518.0
Residual to conserve energy budge [Watts]: 1310004035848.0
